<a href="https://colab.research.google.com/github/ltoosaint24/DS-Unit-2-Linear-Models/blob/master/Loveline_Toussaint_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [2]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [4]:
#Train for April and May 2016 
#Test June 2016


df.sample()



,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
16933,1.0,2,2016-06-01 03:56:57,Amazing Luxury Building prime location in Midt...,W 54 St.,40.7645,-73.984,3250,260 W 54 St.,high,1,1,1,1,1,1,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [5]:
#The datetime object had to be shifted into a datetime data type.
import datetime
datetn = [x for x in df['created']]
for xn in datetn:
  df['created'] =datetime.datetime.strptime(xn,'%Y-%m-%d %H:%M:%S')

In [6]:
df.sample(5)

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
48326,1.0,2,2016-04-12 02:48:07,"Immaculate: Sunny, Lots of Open Space 2 Bedroo...",Upper East Side****as,40.7729,-73.9577,3100,Upper East Side****as,low,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
207,2.0,3,2016-04-12 02:48:07,"Perfect Upper East Side Location, 24 hour door...",E 75th St.,40.7722,-73.9606,8200,166 E 75th St.,low,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
47051,1.0,1,2016-04-12 02:48:07,Renovated Building and Apartment! Be the first...,Mott Street,40.7198,-73.9964,2800,167 Mott Street,low,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
44055,1.0,1,2016-04-12 02:48:07,BEST DEAL OF THE SEASON! Massive 1 bedroom wit...,East 91st Street,40.7824,-73.9521,2600,169 East 91st Street,medium,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22797,2.0,3,2016-04-12 02:48:07,This apartment is situated in a luxury elevato...,East 4th Street,40.7258,-73.9889,5200,94 East 4th Street,high,1,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0


In [7]:
df.dtypes

bathrooms                      float64
bedrooms                         int64
created                 datetime64[ns]
description                     object
display_address                 object
latitude                       float64
longitude                      float64
price                            int64
street_address                  object
interest_level                  object
elevator                         int64
cats_allowed                     int64
hardwood_floors                  int64
dogs_allowed                     int64
doorman                          int64
dishwasher                       int64
no_fee                           int64
laundry_in_building              int64
fitness_center                   int64
pre-war                          int64
laundry_in_unit                  int64
roof_deck                        int64
outdoor_space                    int64
dining_room                      int64
high_speed_internet              int64
balcony                  

In [8]:
from datetime import datetime, date

#created a test and train index for the values based on datetime conditions
train = df[(df['created'] > '2016-04-01 12:00:00') & (df['created'] < '2016-05-31 23:00:00')]
test = df[(df['created'] >= '2016-06-01 12:00:00') & (df['created'] <='2016-06-30 23:00:00')]



In [9]:
#shape of test and train
train.shape, test.shape

((48817, 34), (0, 34))

In [10]:
train

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-04-12 02:48:07,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-04-12 02:48:07,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-12 02:48:07,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-12 02:48:07,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-12 02:48:07,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49347,1.0,2,2016-04-12 02:48:07,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,40.7426,-73.9790,3200,230 E 30 St,medium,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
49348,1.0,1,2016-04-12 02:48:07,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
49349,1.0,1,2016-04-12 02:48:07,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49350,1.0,0,2016-04-12 02:48:07,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
test #There are no matching indexes that fit the specified conditions

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space


In [12]:
#Baseline
train['created'].mean() #average datetime

Timestamp('2016-04-12 02:48:06.998559488')

In [13]:
train['bedrooms'].mean() #average bedrooms

1.5371489440154045

In [14]:
train['bathrooms'].mean() #average bathrooms

1.2017944568490484

In [15]:
test['created'].mean()

NaT

In [16]:
test['bedrooms'].mean()

nan

In [17]:
test['bathrooms'].mean()

nan

In [18]:
target = 'bedrooms'
y_train = train[target]
y_test = test[target]


In [19]:
print('Mean Baseline (using ) features)')
guess = y_train.mean()
guess

Mean Baseline (using ) features)


1.5371489440154045

In [20]:
#Train Error
from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (Bedrooms from April and May 2016): {mae:.2f}  percentage points')

Train Error (Bedrooms from April and May 2016): 0.93  percentage points


In [44]:
#Regression 1 Feature

import pandas as pd
import plotly.express as pax

pax.scatter(
    train,
    x = 'created',
    y= 'bedrooms',
    text= 'created',
    title = "Average number of Bedrooms in Condos From April to May 2016",
  

)

In [22]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()




In [23]:
features = ['bedrooms']
x_train = train[features]
print(f'Linear Regression, dependent on: {features}')


Linear Regression, dependent on: ['bedrooms']


In [37]:
model.fit(x_train, y_train)
y_pred = model.predict(x_train)
mae = mean_absolute_error(y_train,y_pred)
print(f'Train Error: {mae:.2f} percentage points')

Train Error: 0.00 percentage points


In [38]:
model.coef_

array([1.])

In [39]:
model.intercept_

1.7763568394002505e-15

In [42]:
y_pred = model.predict(x_train)
y_pred

array([3.00000000e+00, 2.00000000e+00, 1.00000000e+00, ...,
       1.00000000e+00, 1.77635684e-15, 2.00000000e+00])

In [43]:
#r-score:
from sklearn.metrics import r2_score

r2_score(y_train, y_pred)

1.0

Current score indicates all values are correctly represented at 100%.

In [27]:
import itertools
import numpy as numy
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

def regression_3d(df,x,y,z, num =100, **kwargs):


  fig = px.scatter_3d(df, x, y, z, **kwargs)

  features = [x,y]
  target = z
  model = LinearRegression()
  model.fit(df[features], df[target])

  xmin, xmax = df[x].min(), df[x].max()
  ymin, ymax = df[y].min(), df[y].max()
  xcoords = numy.linspace(xmin, xmax, num)
  ycoords = numy.linspace(ymin, ymax, num)
  coords = list(itertools.product(xcoords, ycoords))

  predictions = model.predict(coords)
  z = predictions.reshape(num, num).T

  fig.add_trace(go.Surface(x= xcoords, y = ycoords, z = Z))

  return fig

In [36]:
regression_3d(
    train,
    x='created',
    y= 'bedrooms',
    z = 'bathrooms',
    text= 'created',
    title = 'Bedrooms created against bathroom'
)


TypeError: ignored